In [152]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import csv
import os

In [153]:
dateStart = '2020-01-01'
dateEnd   = '2020-12-31'

In [154]:
#disable images
option = webdriver.ChromeOptions()
chrome_prefs = {}
option.experimental_options["prefs"] = chrome_prefs
chrome_prefs["profile.default_content_settings"] = {"images": 2}
chrome_prefs["profile.managed_default_content_settings"] = {"images": 2}
driver = webdriver.Chrome(options=option)

In [155]:
driver.get('https://www.hltv.org')
driver.implicitly_wait(15)
accept = driver.find_element_by_link_text('Allow selection')
accept.send_keys(Keys.RETURN)
driver.implicitly_wait(0)

In [156]:
id_start = 20000
id_end = 25000

for id_int in range(id_start-1,id_end):
    playerID = str(id_int+1)
    isEmpty = 0
    isError = 0
    matchesLinks = []
    for offset in ['0','100','200']:
        link = 'https://www.hltv.org/results?content=highlights&'+'player='+playerID+'&startDate='+dateStart+'&endDate='+dateEnd+'&offset='+offset
        driver.get(link)
        try:
            driver.find_element_by_class_name('error-body')
            isError = 1
            break
        except NoSuchElementException:
            True
        elemPag = driver.find_element_by_class_name('pagination-data')
        text = elemPag.text
        a = [int(s) for s in text.split() if s.isdigit()]
        if(a[2] < 40):
            isEmpty = 1
            break
        try:
            driver.find_elements_by_xpath("//*[contains(text(), 'My Button')]")
            elem = driver.find_element_by_css_selector("div.results-all:not(.results-holder)")
            hrefs = elem.find_elements_by_class_name('a-reset')
            for href in hrefs:
                matchesLinks.append(href.get_attribute('href'))
        except NoSuchElementException:
            print('not found')
    if isEmpty == 1 or isError == 1:
        continue
    playerLink = '/player/'+playerID+'/'
    playerName = driver.find_element_by_xpath('//a[contains(@href, "%s")]' % playerLink).text
    print(playerID,' ',playerName,' ', a[2])
    filename = './data/player_parse/matches_' + playerName + '.csv'
    with open(filename, 'a', newline='') as myfile:
        wr = csv.writer(myfile)
        wr.writerow(['links'])
        for link in matchesLinks:
            wr.writerow([link])
driver.close()

not found
not found
20097   h4rn   91
